In [1]:
#ssh sydney
#setPython3 

import numpy 
import numpy as np
import math
from numpy import loadtxt
import pandas as pd

from scipy.io import arff
data = arff.loadarff('scpf.arff')
df = pd.DataFrame(data[0])
df.dropna(inplace=True)
covariables = df.iloc[:,0:23].values
response = df.iloc[:,23:26].values
positions = np.arange(143)

from sklearn.model_selection import train_test_split


covariables_train, covariables_test, response_train, response_test,positions_train,positions_test = train_test_split(covariables, response,positions, test_size=0.20, random_state=42)


##APPRENTISSAGE

def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = numpy.empty(a.shape, dtype=a.dtype)
    shuffled_b = numpy.empty(b.shape, dtype=b.dtype)
    permutation = numpy.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

#import tensorflow as tf 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()



# Import data
# Tensorflow is finicky about shapes, so resize


X_datatrain, X_datatest, targets_train, targets_test = covariables_train, covariables_test, response_train, response_test

#les dimensions de X for training
NbrLignes_train = X_datatrain.shape[0]  ###
NbrColonnes_train= X_datatrain.shape[1] ###

#les dimensions de X for testing
NbrLignes_test = X_datatest.shape[0]  ###
NbrColonnes_test = X_datatest.shape[1] ###
#Y_data = targets 

Y_datatrain = tf.reshape(targets_train, [NbrLignes_train,targets_train.shape[1]])
Y_datatest = tf.reshape(targets_test, [NbrLignes_test,targets_train.shape[1]])

#build the model

# input X: batch_size x NbrColonnes, the first dimension (None) will index the data in the mini-batch
Xfill = tf.placeholder(tf.float32, shape= [None, NbrColonnes_train])
# correct answers will go here
Yfill = tf.placeholder(tf.float32, shape=[None, targets_train.shape[1]]) 
#Let's add some layers

# Probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

# five layers and their number of neurons (tha last layer has 10 softmax neurons)
L = 10 #200
M = 3 #160
#N = 100
#O = 30

# Weights initialised with small random values between -0.2 and +0.2
# When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
W1 = tf.Variable(tf.truncated_normal([NbrColonnes_train, L], stddev=0.1))  # 
B1 = tf.Variable(tf.ones([L])/10)
W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.ones([M])/10)
#W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
#B3 = tf.Variable(tf.ones([N])/10)
#W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
#B4 = tf.Variable(tf.ones([O])/10)
#W5 = tf.Variable(tf.truncated_normal([O, 6], stddev=0.1))
#B5 = tf.Variable(tf.zeros([6]))

# The model, with dropout at each layer


Y1 = tf.nn.relu(tf.matmul(Xfill, W1) + B1)
Y1d = tf.nn.dropout(Y1, pkeep)

#Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
#Y2d = tf.nn.dropout(Y2, pkeep)

#Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
#Y3d = tf.nn.dropout(Y3, pkeep)

#Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
#Y4d = tf.nn.dropout(Y4, pkeep)

#Y_ = tf.matmul(Y4d, W5) + B5


Y_ = tf.matmul(Y1d, W2) + B2

cross_entropy = tf.losses.mean_squared_error(Yfill, Y_)

# accuracy of the trained model, between 0 (worst) and 1 (best)
#Modification de l'accuracy 

#calcul de l'accuracy, une nouvelle manière: :  

# variable learning rate
lr = tf.placeholder(tf.float32)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy) 

# init
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)


max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 100.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations

train_loss = []
test_loss = []

# Gradient descent loop for 500 steps
for i in range(3500):
 # Select random minibatch
    j = 1
    ##### shuffling our data::: really important
    X_datasf, Y_datasf = shuffle_in_unison(X_datatrain, Y_datatrain.eval())
    
    ##### defining a decreasing learning rate: 
    # learning rate decay
    learning_rate = 0.0001001329829992624
    #learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
    
    ##### 
    for start, end in zip(range(0, NbrLignes_train, 52), range(20, NbrLignes_train, 52)):

             X_batch,Y_batch = X_datasf[start:end,],Y_datasf[start:end,]
             j +=1
             # Do gradient descent step
             _, loss_val_train = sess.run([train_step, cross_entropy], feed_dict={Xfill: X_batch, Yfill: Y_batch, pkeep: 0.75, lr: learning_rate})
             if i %20==0:    
                 
             
                #if j%20==0:
                 loss_val_test = sess.run(cross_entropy, feed_dict={Xfill: X_datatest, Yfill: Y_datatest.eval(), pkeep: 1.0, lr:  learning_rate})
                 train_loss.append(loss_val_train)
                 
                 test_loss.append(loss_val_test)
                 print('LOSS at step %s: - Train loss value : %f -- Test loss value : %f' % (i, loss_val_train,loss_val_test))
#print(test_loss)
#print(test_loss)
#LOSS at step 420: - Train loss value : 22.385496 -- Test loss value : 9.721767

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
LOSS at step 0: - Train loss value : 1154.701782 -- Test loss value : 683.128479
LOSS at step 0: - Train loss value : 476.647430 -- Test loss value : 682.931396
LOSS at step 20: - Train loss value : 749.076294 -- Test loss value : 676.307800
LOSS at step 20: - Train loss value : 1195.392212 -- Test loss value : 676.151733
LOSS at step 40: - Train loss value : 857.700073 -- Test loss value : 670.101440
LOSS at step 40: - Train loss value : 526.549072 -- Test loss value : 669.959656
LOSS at step 60: - Train loss value : 899.596069 -- Test loss value : 664.068359
LOSS at step 60: - Train loss value : 1102.836060 -- Test loss value : 663.919250
LOSS at step 80: - Train loss value : 15489.024414 -- Test loss value : 657.730103
LOSS at step 80: - Train loss value : 726.981873 -- Test loss value 

LOSS at step 920: - Train loss value : 409.040497 -- Test loss value : 390.003632
LOSS at step 940: - Train loss value : 174.143356 -- Test loss value : 390.426727
LOSS at step 940: - Train loss value : 522.111145 -- Test loss value : 390.438293
LOSS at step 960: - Train loss value : 767.118713 -- Test loss value : 390.904510
LOSS at step 960: - Train loss value : 781.696472 -- Test loss value : 390.916260
LOSS at step 980: - Train loss value : 104.071175 -- Test loss value : 391.443909
LOSS at step 980: - Train loss value : 14498.909180 -- Test loss value : 391.491150
LOSS at step 1000: - Train loss value : 488.078125 -- Test loss value : 392.645416
LOSS at step 1000: - Train loss value : 716.452148 -- Test loss value : 392.656982
LOSS at step 1020: - Train loss value : 705.306091 -- Test loss value : 393.054718
LOSS at step 1020: - Train loss value : 813.453369 -- Test loss value : 393.089294
LOSS at step 1040: - Train loss value : 301.585999 -- Test loss value : 394.042847
LOSS at s

LOSS at step 1960: - Train loss value : 364.991486 -- Test loss value : 404.249359
LOSS at step 1960: - Train loss value : 14237.113281 -- Test loss value : 404.292755
LOSS at step 1980: - Train loss value : 680.628052 -- Test loss value : 404.014862
LOSS at step 1980: - Train loss value : 222.194397 -- Test loss value : 403.987823
LOSS at step 2000: - Train loss value : 642.269104 -- Test loss value : 404.099091
LOSS at step 2000: - Train loss value : 421.417786 -- Test loss value : 404.096863
LOSS at step 2020: - Train loss value : 294.027710 -- Test loss value : 404.549377
LOSS at step 2020: - Train loss value : 579.331238 -- Test loss value : 404.566376
LOSS at step 2040: - Train loss value : 212.068665 -- Test loss value : 404.519806
LOSS at step 2040: - Train loss value : 196.527740 -- Test loss value : 404.477509
LOSS at step 2060: - Train loss value : 329.053375 -- Test loss value : 403.771790
LOSS at step 2060: - Train loss value : 445.371948 -- Test loss value : 403.722382
LO

LOSS at step 3000: - Train loss value : 714.054382 -- Test loss value : 401.293854
LOSS at step 3000: - Train loss value : 185.861679 -- Test loss value : 401.294403
LOSS at step 3020: - Train loss value : 233.895218 -- Test loss value : 402.796997
LOSS at step 3020: - Train loss value : 157.075714 -- Test loss value : 402.810150
LOSS at step 3040: - Train loss value : 171.287140 -- Test loss value : 402.286865
LOSS at step 3040: - Train loss value : 614.005920 -- Test loss value : 402.272980
LOSS at step 3060: - Train loss value : 502.615509 -- Test loss value : 402.584045
LOSS at step 3060: - Train loss value : 760.342896 -- Test loss value : 402.607635
LOSS at step 3080: - Train loss value : 280.301483 -- Test loss value : 403.737976
LOSS at step 3080: - Train loss value : 226.556747 -- Test loss value : 403.766602
LOSS at step 3100: - Train loss value : 242.095001 -- Test loss value : 403.401855
LOSS at step 3100: - Train loss value : 714.443298 -- Test loss value : 403.340515
LOSS

# Let us compute the aRRMSE

In [2]:
learning_rate = 0.0001001329829992624

Y_testpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatest, tf.float32).eval(), Yfill: tf.cast(Y_datatest, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})
#numpy.savetxt('Y_testpred_with_stock_%s_and_ai_%s_modelcut_%s_vartimeframe_%s.txt'%(stock, ai, cut, vartimeframe), Y_testpred, fmt='%f')
Y_trainpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatrain, tf.float32).eval(), Yfill: tf.cast(Y_datatrain, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})


In [3]:
Y_testpred

array([[31.32908   ,  2.0517406 ,  0.37311298],
       [50.756817  ,  3.0611475 ,  0.49672794],
       [46.503292  ,  2.8344946 ,  0.46529433],
       [43.17923   ,  2.6629682 ,  0.4450575 ],
       [44.60006   ,  2.7436042 ,  0.45936495],
       [32.598854  ,  2.1172276 ,  0.38081616],
       [29.939213  ,  1.9880545 ,  0.37086087],
       [35.061325  ,  2.2522986 ,  0.40199453],
       [34.12699   ,  2.1958644 ,  0.3899514 ],
       [43.69701   ,  2.6971931 ,  0.45401222],
       [38.10405   ,  2.4011838 ,  0.41423917],
       [27.238756  ,  1.848175  ,  0.35400942],
       [35.196274  ,  2.2512095 ,  0.39659166],
       [36.5485    ,  2.320959  ,  0.40480283],
       [34.335625  ,  2.2138534 ,  0.39680508],
       [34.707607  ,  2.2332668 ,  0.39923874],
       [34.101562  ,  2.2021065 ,  0.39563605],
       [33.747128  ,  2.1765046 ,  0.38782564],
       [29.20662   ,  1.9496484 ,  0.3659347 ],
       [31.26982   ,  2.0485868 ,  0.37267816],
       [36.34099   ,  2.3102508 ,  0.403

In [4]:
from numpy import linalg as LA

In [5]:

aRRMSE_list = []

for i in range(Y_testpred.shape[1]) :
    
    Errors_test =  Y_testpred[:,i] - response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    ytrain_mean_i = np.tile(Y_trainpred[:,i].mean(axis = 0), (covariables_test.shape[0], 1))

    Errors_relative = ytrain_mean_i - response_test[:,i]

    Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%Final_SPNNR_aRRMSE)

    aRRMSE_list.append(Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 18.080463
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 16.365589
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 18.142866


In [6]:
np.mean(aRRMSE_list)

17.529639547931865

In [9]:
#Taking aRRMSE on PCA or inverted PCA. ?

# Check if old way calculation is wrong

In [10]:
Errors_test =  Y_testpred - response_test


print('<*********************************Let us check the aRRMSE*********************************>')

ytrain_mean = np.tile(Y_trainpred.mean(axis = 0), (response_test.shape[0], 1))

Errors_relative = ytrain_mean - response_test

Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

print("Final_Partition aRRMSE is : %f"%Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final_Partition aRRMSE is : 82.647642


*** The answer is yes ***

# Checking True aRRMSE of S-SPNNR on Slump

In [11]:
spnnr_covariables_train, spnnr_covariables_test, spnnr_response_train, spnnr_response_test,spnnr_positions_train,spnnr_positions_test = train_test_split(covariables, response,positions, test_size=0.33, random_state=42)

In [12]:
spnnr_Y_testpred =np.loadtxt('All_Y_test_predictions.out', delimiter=',' )
spnnr_Y_trainpred =np.loadtxt('All_Y_train_predictions.out', delimiter=',' )

In [10]:
spnnr_aRRMSE_list = []

for i in range(spnnr_Y_testpred.shape[1]) :
    
    spnnr_Errors_test =  spnnr_Y_testpred[:,i] - spnnr_response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    spnnr_ytrain_mean_i = np.tile(spnnr_Y_trainpred[:,i].mean(axis = 0), (spnnr_covariables_test.shape[0], 1))

    spnnr_Errors_relative = spnnr_ytrain_mean_i - spnnr_response_test[:,i]

    spnnr_Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(spnnr_Errors_test)**2)/(LA.norm(spnnr_Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%spnnr_Final_SPNNR_aRRMSE)

    spnnr_aRRMSE_list.append(spnnr_Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 9.212108
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 17.784301
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 18.862991


In [11]:
np.mean(spnnr_aRRMSE_list)

15.286466834678842